In [ ]:
# Song Recommender

# Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt


In [9]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pprint
from time import sleep
from config import *

In [2]:
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
pd.set_option("display.max_columns",None)

In [4]:
%matplotlib inline


# User Inputs a Song

In [6]:
user_song = input('Please enter the name of a song: ')

Please enter the name of a song: Aqui no hay playa


In [7]:
user_artist = input('Please enter the artist of the song: ')

Please enter the artist of the song: The refrescos


## User input function

In [109]:
def inputSong():
    """ Prompt the user for a song and an artist"""
    
    user_song = ''
    artist_song = ''
    
    while True:
        user_song = input('Please enter the name of a song: ')
        user_song = user_song.replace("'",'').title()
        
        user_artist = input('Please enter the artist of the song: ')
        user_artist = user_artist.replace("'",'').title()
        
        print()
        print('Your song is: %s' %(user_song))
        print('The artist of the song is: %s' %(user_artist))
        
        command = input('\nIf your input song and artist are right, please press enter to continue. \n\
If they are not right, press any key to enter them again.')
        
        if len(command) == 0:
            break
        else:
            continue
            
    return user_song, user_artist
    
    

In [53]:
inputSong()

Please enter the name of a song: aqui no hay playa
Please enter the artist of the song: the refrescos

Your song is: Aqui No Hay Playa
The artist of the song is: The Refrescos

If your input song and artist are right press enter to continue. 
If not press any key to enter them again.


# Get user's song audio features

In [10]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret))

In [28]:
user_song_df = querySong()
user_song_df

Querying IDs...
Querying IDs...Done.


,song,artist,href,id,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Aqui no hay playa,The refrescos,https://api.spotify.com/v1/tracks/4PRJD9T23GQK...,4PRJD9T23GQKQYKhJQcGGY,spotify:track:4PRJD9T23GQKQYKhJQcGGY,0.583,0.949,9,-8.121,1,0.0385,0.277,0.00175,0.144,0.954,168.178,audio_features,4PRJD9T23GQKQYKhJQcGGY,spotify:track:4PRJD9T23GQKQYKhJQcGGY,https://api.spotify.com/v1/tracks/4PRJD9T23GQK...,https://api.spotify.com/v1/audio-analysis/4PRJ...,217627,4


In [162]:
def querySong(song = user_song, artist = user_artist):
    """ search the ids and audio features of a song"""
    
    print('Querying IDs...')

    user_song_id = {'song': [], 'artist': [], 'album': [], 'href':[],'id':[],'uri':[]}
    #song_validation = pd.DataFrame()

    result = sp.search(q=r"track:{} artist:{}".format(song,artist), limit=5)


    if result['tracks']['total']>0:        
        tracks = result['tracks']['total']

        if tracks > 5:
            tracks = 5

        
        # Querying ids
        for x in range(tracks):
            user_song_id['href'].append(result['tracks']['items'][x]['href'])
            user_song_id['id'].append(result['tracks']['items'][x]['id'])
            user_song_id['uri'].append(result['tracks']['items'][x]['uri'])
            user_song_id['album'].append(result['tracks']['items'][x]['album']['name'])
            user_song_id['artist'].append(result['tracks']['items'][x]['artists'][0]['name'])
            user_song_id['song'].append(result['tracks']['items'][x]['name'])
            
            
        song_validation = pd.DataFrame(user_song_id)
            
        print('Querying IDs...Done.')
        
        # Song Validation
        
        while True:
            display(song_validation) #[['song','artist','album']])
        
            idx = input('Please confirm the version of your song by introducing the index (0-4):')
            
            try:
                idx = int(idx)
            except:
                print('Please use numeric digits.')
                continue
            
            if idx < 0 or idx > 4:
                print('Please enter an integer number in the range [0,4].')
                continue
            break
        
        # display(song_validation.iloc[idx])
        
        user_song_id = pd.DataFrame(song_validation.iloc[idx]).T.reset_index(drop=True) 
        # display(user_song_id)
        
        # Quering audio features
        
        #Get Audio Features
        audio_feat = sp.audio_features(user_song_id['uri'])[0]
        print('Querying audio features...Done.')
        
        #Define the empty dictionary
        audio_dict = { key : [] for key in list(audio_feat.keys()) }
        
        # Create the dictionary for the dataframe
        audio_dict = { key : audio_dict[key] + [audio_feat[key]] for key in list(audio_dict.keys())}
        # pprint.pprint(audio_dict)

        
    else:
        print('There was not found that song. Please try again.')
    

    
    # Process output 
    # That output processing is returning error when no song is found.
    #user_song_id = pd.DataFrame(user_song_id)    
    
    user_song_audio = pd.DataFrame(audio_dict)
    user_song_features = pd.concat([user_song_id,user_song_audio], axis=1)
    
    
    return user_song_features



In [137]:
querySong()

Querying IDs...
Querying IDs...Done.


,song,artist,album,href,id,uri
0,Aquí No Hay Playa,The Refrescos,The Refrescos,https://api.spotify.com/v1/tracks/4PRJD9T23GQK...,4PRJD9T23GQKQYKhJQcGGY,spotify:track:4PRJD9T23GQKQYKhJQcGGY
1,Aquí No Hay Playa,The Refrescos,Pongamos Que Hablo de Madrid,https://api.spotify.com/v1/tracks/2Qjq1oKHQCSn...,2Qjq1oKHQCSno0pScYnbeB,spotify:track:2Qjq1oKHQCSno0pScYnbeB
2,Aquí no hay playa,The Refrescos,Let´s Ska!,https://api.spotify.com/v1/tracks/1d2TSO9TqkJ6...,1d2TSO9TqkJ6dTttldj7kG,spotify:track:1d2TSO9TqkJ6dTttldj7kG
3,Aquí No Hay Playa,The Refrescos,Me Quedo con los Clásicos,https://api.spotify.com/v1/tracks/2wtpI6zJwRtO...,2wtpI6zJwRtO7hC9pODKBq,spotify:track:2wtpI6zJwRtO7hC9pODKBq
4,Aquí No Hay Playa,The Refrescos,Clásicos de Lujo,https://api.spotify.com/v1/tracks/2cBO5lsHywaE...,2cBO5lsHywaEJZEZqDtPgO,spotify:track:2cBO5lsHywaEJZEZqDtPgO


Please confirm the version of your song by introducing the index (0-4):2


song                                      Aquí no hay playa
artist                                        The Refrescos
album                                            Let´s Ska!
href      https://api.spotify.com/v1/tracks/1d2TSO9TqkJ6...
id                                   1d2TSO9TqkJ6dTttldj7kG
uri                    spotify:track:1d2TSO9TqkJ6dTttldj7kG
Name: 2, dtype: object

,song,artist,album,href,id,uri
0,Aquí no hay playa,The Refrescos,Let´s Ska!,https://api.spotify.com/v1/tracks/1d2TSO9TqkJ6...,1d2TSO9TqkJ6dTttldj7kG,spotify:track:1d2TSO9TqkJ6dTttldj7kG


Querying audio features...Done.


,song,artist,album,href,id,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Aquí no hay playa,The Refrescos,Let´s Ska!,https://api.spotify.com/v1/tracks/1d2TSO9TqkJ6...,1d2TSO9TqkJ6dTttldj7kG,spotify:track:1d2TSO9TqkJ6dTttldj7kG,0.464,0.795,9,-10.175,1,0.115,0.206,0.0835,0.511,0.545,174.257,audio_features,1d2TSO9TqkJ6dTttldj7kG,spotify:track:1d2TSO9TqkJ6dTttldj7kG,https://api.spotify.com/v1/tracks/1d2TSO9TqkJ6...,https://api.spotify.com/v1/audio-analysis/1d2T...,570607,4


# Predit user song cluster

In [176]:
def predictCluster(user_df):
    """It predicts the cluster for the user song"""
    
    # Load the scaler
    with open('scalers/stdScaler_20220610.pickle', "rb") as file:
        scaler = pickle.load(file)
        
    # Load the model
    with open('models/kmeans_20220610_12.pickle', "rb") as file:
        best_clustering = pickle.load(file)
        
    
    # Scale audio features
    X = pd.concat([user_df.loc[:,'danceability':'tempo'],user_df.loc[:,'duration_ms':'time_signature']],axis=1)
    display(X)
    X_scaled = scaler.transform(X)
    # Save X_scaled as dataframe
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    display(X_scaled)
    
    # Predict cluster
    cluster = best_clustering.predict(X_scaled)
    
    
    # Process output
    user_df['cluster'] = cluster
    
    return user_df
    
    

In [67]:
user_song_df = predictCluster(user_song_df)
user_song_df

/Users/albertomengual/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/albertomengual/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KMeans from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.583,0.949,9,-8.121,1,0.0385,0.277,0.00175,0.144,0.954,168.178,217627


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.201562,1.294046,1.033059,0.283574,0.711146,-0.426444,-0.108661,-0.630948,-0.360352,1.670814,1.636055,-0.252499


,song,artist,href,id,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
0,Aqui no hay playa,The refrescos,https://api.spotify.com/v1/tracks/4PRJD9T23GQK...,4PRJD9T23GQKQYKhJQcGGY,spotify:track:4PRJD9T23GQKQYKhJQcGGY,0.583,0.949,9,-8.121,1,0.0385,0.277,0.00175,0.144,0.954,168.178,audio_features,4PRJD9T23GQKQYKhJQcGGY,spotify:track:4PRJD9T23GQKQYKhJQcGGY,https://api.spotify.com/v1/tracks/4PRJD9T23GQK...,https://api.spotify.com/v1/audio-analysis/4PRJ...,217627,4,0


In [39]:
user_song_df.track_href

0    https://api.spotify.com/v1/tracks/4PRJD9T23GQK...
Name: track_href, dtype: object

# Song recommender

In [165]:
# Load the songs' dataframe
full = pd.read_csv('data/full_20220610.csv')

In [166]:
full.head()

,Unnamed: 0,song,singer,id,url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,hot,cluster
0,0,As It Was,Harry Styles,4LRPiXqCikLlN15c3yImP7,https://open.spotify.com/track/4LRPiXqCikLlN15...,0.520,0.731,6.0,-5.338,0.0,0.0557,0.3420,0.00101,0.3110,0.662,173.930,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303.0,4.0,Yes,6
1,1,First Class,Jack Harlow,1rDQ4oMwGJI7B4tovsBOxc,https://open.spotify.com/track/1rDQ4oMwGJI7B4t...,0.905,0.563,8.0,-6.135,1.0,0.1020,0.0254,0.00001,0.1130,0.324,106.998,audio_features,1rDQ4oMwGJI7B4tovsBOxc,spotify:track:1rDQ4oMwGJI7B4tovsBOxc,https://api.spotify.com/v1/tracks/1rDQ4oMwGJI7...,https://api.spotify.com/v1/audio-analysis/1rDQ...,173948.0,4.0,Yes,11
2,2,Wait For U,Future Featuring Drake & Tems,59nOXPmaKlBfGMDeOVGrIK,https://open.spotify.com/track/59nOXPmaKlBfGMD...,0.463,0.642,1.0,-4.474,1.0,0.3400,0.3140,0.00000,0.0686,0.339,83.389,audio_features,59nOXPmaKlBfGMDeOVGrIK,spotify:track:59nOXPmaKlBfGMDeOVGrIK,https://api.spotify.com/v1/tracks/59nOXPmaKlBf...,https://api.spotify.com/v1/audio-analysis/59nO...,189893.0,4.0,Yes,1
3,3,Late Night Talking,Harry Styles,1qEmFfgcLObUfQm0j1W2CK,https://open.spotify.com/track/1qEmFfgcLObUfQm...,0.714,0.728,10.0,-4.595,1.0,0.0468,0.2980,0.00000,0.1060,0.901,114.996,audio_features,1qEmFfgcLObUfQm0j1W2CK,spotify:track:1qEmFfgcLObUfQm0j1W2CK,https://api.spotify.com/v1/tracks/1qEmFfgcLObU...,https://api.spotify.com/v1/audio-analysis/1qEm...,177955.0,4.0,Yes,11
4,4,About Damn Time,Lizzo,1PckUlxKqWQs3RlWXVBLw3,https://open.spotify.com/track/1PckUlxKqWQs3Rl...,0.836,0.743,10.0,-6.305,0.0,0.0656,0.0995,0.00000,0.3350,0.722,108.966,audio_features,1PckUlxKqWQs3RlWXVBLw3,spotify:track:1PckUlxKqWQs3RlWXVBLw3,https://api.spotify.com/v1/tracks/1PckUlxKqWQs...,https://api.spotify.com/v1/audio-analysis/1Pck...,191822.0,4.0,Yes,2


In [201]:
list(full.columns) # .index("url")

['Unnamed: 0',
 'song',
 'singer',
 'id',
 'url',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'id.1',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'hot',
 'cluster']

In [202]:
def songRecommender(user_df):
    """ Recommends a song based on the input song"""
    
    # Load the songs' dataframe
    full = pd.read_csv('data/full_20220610.csv')
    
    # User song's cluster
    user_cluster = user_df.cluster[0]
    print('user cluster is: %s' %(user_cluster))
    
    # Check if it is in Hot data base
    if full.song[full.hot == 'Yes'].isin(user_df.song).sum() > 0:
        recommendation = full[(full.hot == 'Yes') & (full.cluster == user_cluster)].sample()
    else:
        recommendation = full[(full.hot == 'No') & (full.cluster == user_cluster)].sample()
        
    # Print the result
    print('The recommended song is:')
    print(recommendation.iloc[0,1]) #song)
    print(recommendation.iloc[0,2]) #singer)
    print(recommendation.iloc[0,4]) #full.loc[0,"url"]
    
    return recommendation #[['song','singer','url']]
        

In [78]:
user_clus = user_song_df.cluster[0]
user_clus

0

In [76]:
type(user_clus[0])

numpy.int32

In [79]:
full[(full.hot == 'Yes') & (full.cluster == user_clus)].sample()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,song,singer,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,hot,cluster
75,75,75,75,Dua Lipa,Jack Harlow,0LnS7aOdOdI1dNKZqdOLz4,0.833,0.652,11.0,-6.942,1.0,0.0762,0.00161,0.0985,0.106,0.405,158.022,audio_features,0LnS7aOdOdI1dNKZqdOLz4,spotify:track:0LnS7aOdOdI1dNKZqdOLz4,https://api.spotify.com/v1/tracks/0LnS7aOdOdI1...,https://api.spotify.com/v1/audio-analysis/0LnS...,135053.0,4.0,Yes,0


In [73]:
type(full.cluster[1])

numpy.int64

In [83]:
songRecommender(user_song_df)

user cluster is: 0
The recommended song is:
3573    Trapped
Name: song, dtype: object
3573    Colonel Abrams
Name: singer, dtype: object


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,song,singer,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,hot,cluster
3573,4853,4853,4853,Trapped,Colonel Abrams,6NMtQazEdK3sjnTISmzmXS,0.794,0.823,9.0,-8.227,1.0,0.0408,0.0427,0.000478,0.119,0.872,115.29,audio_features,6NMtQazEdK3sjnTISmzmXS,spotify:track:6NMtQazEdK3sjnTISmzmXS,https://api.spotify.com/v1/tracks/6NMtQazEdK3s...,https://api.spotify.com/v1/audio-analysis/6NMt...,389533.0,4.0,No,0


# The general song recommender function

In [203]:
def recommend():
    """ Ask the user to input a song and then recommends a similar one"""
    
    recommended_songs = pd.DataFrame()
    
    next_recommend = True
    
    while next_recommend:
        # Prompt user input
        user_song, user_artist = inputSong()

        #Initialize SpotiPy with user credentias
        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                               client_secret= client_secret))


        # Get user's song's features
        try:
            user_song_features = querySong(user_song, user_artist)
        except:
            print('There was not found that song. Please try again.')
            continue


        # Predict user's song's cluster
        user_song_features = predictCluster(user_song_features)

        # Get a recommended song
        recommendation = songRecommender(user_song_features)
        
        display(recommendation[['song','singer','url']])
        
        # Process output
        recommended_songs = pd.concat([recommended_songs,recommendation], axis=0)
        
        # Ask for another recommendation:
        while True:
            another = input('Do you need another recommendation? (Y - Yes, N - No)')
            
            if another.lower() in ['y', 'yes']:
                break
            elif another.lower() in ['n', 'no']:
                next_recommend = False
                break
            else:
                continue
    
    return recommended_songs
        
        
    
    

In [204]:
recommend()

Please enter the name of a song: help
Please enter the artist of the song: the beatles

Your song is: Help
The artist of the song is: The Beatles

If your input song and artist are right, please press enter to continue. 
If they are not right, press any key to enter them again.
Querying IDs...
Querying IDs...Done.


,song,artist,album,href,id,uri
0,Help! - Remastered 2009,The Beatles,Help! (Remastered),https://api.spotify.com/v1/tracks/7DD7eSuYSC5x...,7DD7eSuYSC5xk2ArU62esN,spotify:track:7DD7eSuYSC5xk2ArU62esN
1,With A Little Help From My Friends - Remastere...,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,https://api.spotify.com/v1/tracks/2RnPATK99oGO...,2RnPATK99oGOZygnD2GTO6,spotify:track:2RnPATK99oGOZygnD2GTO6
2,Help! - Remastered 2015,The Beatles,1 (Remastered),https://api.spotify.com/v1/tracks/3Smida2eCUsL...,3Smida2eCUsLzDcmZqXEZ3,spotify:track:3Smida2eCUsLzDcmZqXEZ3
3,Help!,The Beatles,Love,https://api.spotify.com/v1/tracks/6EdSN1iGtLPh...,6EdSN1iGtLPhcz43QDRkdK,spotify:track:6EdSN1iGtLPhcz43QDRkdK
4,With A Little Help From My Friends - Remix,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Super D...,https://api.spotify.com/v1/tracks/08XKCwMrwEAy...,08XKCwMrwEAyyfTCTGkkhg,spotify:track:08XKCwMrwEAyyfTCTGkkhg


Please confirm the version of your song by introducing the index (0-4):2
Querying audio features...Done.


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.602,0.736,11,-6.51,0,0.0332,0.149,0,0.0776,0.819,94.931,139240,4


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.222537,0.424188,1.612516,0.528131,-1.504488,-0.479349,-0.408373,-0.522934,-0.706599,1.140217,-0.935394,-1.100079,0.208613


user cluster is: 2
The recommended song is:
What Shall We Do Next?
Club 8
https://open.spotify.com/track/7qf2Yl9TE34Q0Ur8J50uvk


,song,singer,url
2744,What Shall We Do Next?,Club 8,https://open.spotify.com/track/7qf2Yl9TE34Q0Ur...


Do you need another recommendation? (Y - Yes, N - No)n


,Unnamed: 0,song,singer,id,url,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id.1,uri,track_href,analysis_url,duration_ms,time_signature,hot,cluster
2744,3444,What Shall We Do Next?,Club 8,7qf2Yl9TE34Q0Ur8J50uvk,https://open.spotify.com/track/7qf2Yl9TE34Q0Ur...,0.664,0.65,1.0,-7.825,0.0,0.0361,0.131,0.0123,0.139,0.351,104.969,audio_features,7qf2Yl9TE34Q0Ur8J50uvk,spotify:track:7qf2Yl9TE34Q0Ur8J50uvk,https://api.spotify.com/v1/tracks/7qf2Yl9TE34Q...,https://api.spotify.com/v1/audio-analysis/7qf2...,156867.0,4.0,No,2


In [192]:
full.loc[0,"url"]

'https://open.spotify.com/track/4LRPiXqCikLlN15c3yImP7'